In [1]:
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer

# Implemented by myself
from config import *
from data_processer import CSCDataset, split_torch_dataset
from models import CombineBertModel, DecoderBaseRNN, DecoderTransformer, Trainer

/root/anaconda3/envs/simple_csc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [3]:
train_dataset = CSCDataset([SIGHAN_train_dir_err, SIGHAN_train_dir_corr], tokenizer)

test_dataset = CSCDataset([SIGHAN_train_dir_err14, SIGHAN_train_dir_corr14], tokenizer)

preprocessing sighan dataset: 2339it [00:00, 900208.94it/s]
preprocessing sighan dataset: 100%|██████████| 2339/2339 [00:00<00:00, 1338948.69it/s]


共2339句，共73264字，最长的句子有171字


preprocessing sighan dataset: 3437it [00:00, 847889.83it/s]
preprocessing sighan dataset: 100%|██████████| 3437/3437 [00:00<00:00, 1356145.14it/s]

共3437句，共170330字，最长的句子有258字


In [4]:
train_data, dev_data = split_torch_dataset(train_dataset, 0.3)

train_data_loader = DataLoader(train_data, num_workers=4, shuffle=True, batch_size=16)

dev_data_loader = DataLoader(dev_data, num_workers=4, shuffle=True, batch_size=16)

test_data_loader = DataLoader(test_dataset, num_workers=4, shuffle=True, batch_size=32)

In [5]:
epochs = 35

# tokenizer = BertTokenizer.from_pretrained(checkpoint)
encoder_model = BertModel.from_pretrained(checkpoint)

# The Hyperparameters can be defined in config.py
hidden_size = 1024
num_layers = 2

decoder_model = DecoderBaseRNN(
    model=nn.LSTM,
    input_size=encoder_model.config.hidden_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

optimizer = AdamW(model.parameters(), lr=learning_rate)
trainer = Trainer(model=model, tokenizer=tokenizer, optimizer=optimizer)

In [6]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

train Epoch:1/35: 100%|██████████| 103/103 [00:28<00:00,  3.62it/s, loss=5.974]

Epoch 1 Loss: 7.091787435476062



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.95it/s, loss=5.753]

Epoch 1 Loss: 5.790225950154391
5.790225950154391 {'over_corr': 20517, 'total_err': 952, 'true_corr': tensor(0, device='cuda:0')} {'over_corr': 701, 'total_err': 701, 'true_corr': 0}



train Epoch:2/35: 100%|██████████| 103/103 [00:28<00:00,  3.64it/s, loss=5.533]

...

Epoch 35 Loss: 0.5563845376945237



dev Epoch:1/1: 100%|██████████| 44/44 [00:07<00:00,  5.58it/s, loss=0.568]

Epoch 1 Loss: 0.7631391408768567
0.7631391408768567 {'over_corr': 1289, 'total_err': 952, 'true_corr': tensor(297, device='cuda:0')} {'over_corr': 468, 'total_err': 701, 'true_corr': 73}



dev Epoch:1/1: 100%|██████████| 108/108 [00:37<00:00,  2.91it/s, loss=0.766]

Epoch 1 Loss: 0.9268376799645247
0.9268376799645247 {'over_corr': 14167, 'total_err': 5278, 'true_corr': tensor(1205, device='cuda:0')} {'over_corr': 3004, 'total_err': 3436, 'true_corr': 107}


In [8]:
encoder_model = BertModel.from_pretrained(checkpoint)

nhead = 2
num_encoder_layers = 2
num_decoder_layers = 2

decoder_model = DecoderTransformer(
    input_size=encoder_model.config.hidden_size,
    nhead=nhead,
    num_encoder_layers=num_encoder_layers,
    num_decoder_layers=num_decoder_layers,
)

model = CombineBertModel(encoder_model=encoder_model, decoder_model=decoder_model)

In [9]:
trainer.train(
    dataloader=train_data_loader, epoch=epochs, test_dataloader=dev_data_loader
)
trainer.test(test_data_loader)

train Epoch:1/35: 100%|██████████| 103/103 [00:28<00:00,  3.63it/s, loss=0.392]

Epoch 1 Loss: 0.512317058530826



dev Epoch:1/1: 100%|██████████| 44/44 [00:08<00:00,  5.47it/s, loss=0.735]

Epoch 1 Loss: 0.7329817278818651
0.7329817278818651 {'over_corr': 1204, 'total_err': 952, 'true_corr': tensor(312, device='cuda:0')} {'over_corr': 459, 'total_err': 701, 'true_corr': 84}



train Epoch:2/35: 100%|██████████| 103/103 [00:28<00:00,  3.62it/s, loss=0.454]

...

Epoch 35 Loss: 0.04083291152763425



dev Epoch:1/1: 100%|██████████| 44/44 [00:08<00:00,  5.19it/s, loss=0.480]

Epoch 1 Loss: 0.44386737048625946
0.44386737048625946 {'over_corr': 332, 'total_err': 952, 'true_corr': tensor(309, device='cuda:0')} {'over_corr': 201, 'total_err': 701, 'true_corr': 167}



dev Epoch:1/1: 100%|██████████| 108/108 [00:39<00:00,  2.75it/s, loss=0.707]

Epoch 1 Loss: 0.5030088256354686
0.5030088256354686 {'over_corr': 3781, 'total_err': 5278, 'true_corr': tensor(1238, device='cuda:0')} {'over_corr': 1692, 'total_err': 3436, 'true_corr': 384}
